In [2]:
from datetime import datetime, timedelta
from pathlib import Path
import os
import sys
import numpy as np
import pandas as pd
import re
import AMBRA_Backups
from bs4 import BeautifulSoup
import AMBRA_Utils


db_name = 'CAPTIVA_MRI'
db = AMBRA_Backups.database.Database(db_name)
project = AMBRA_Backups.redcap_funcs.get_redcap_project('CAPTIVA MRI DATA COLLECTION')

def q(query, record=None):
    if any(substring in query for substring in ['UPDATE', 'INSERT', 'DELETE']):
        return db.run_insert_query(query, record)
    elif 'SELECT' in query:
        return pd.DataFrame(db.run_select_query(query, record, column_names=True))
    else:
        print('You dont have UPDATE, INSERT, DELETE or SELECT in your query my guy')

In [7]:
crf_name = 'vwmri_crf'

schema_questions = pd.DataFrame(db.run_select_query("""SELECT DISTINCT(question_text) FROM CRF_Schema_RedCap
                                WHERE crf_name = %s AND question_text IS NOT NULL""", [crf_name], column_names=True))['question_text']

api_questions = pd.DataFrame(project.metadata)
def only_html(row):
    soup = BeautifulSoup(row['field_label'], 'html.parser')
    if bool(soup.find()):
        return row['field_label']
master_html = ''.join(api_questions.apply(only_html, axis=1).dropna().values.tolist())


api_questions = api_questions[(api_questions['form_name'] == crf_name) &
                            (api_questions['field_type'] != 'descriptive') &
                            (~api_questions['field_name'].apply(lambda x: x in master_html))][['field_name','field_label']]
question_discreps = api_questions[~api_questions['field_label'].isin(schema_questions)]
ques_discrep_string = ''
if not question_discreps.empty:
    discrep_dict = {v[0]:v[1] for v in question_discreps.values}
    ques_discrep_string = f"\nThe following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:\n{{redcap_variable : question_text}}\n\n{discrep_dict}\n\n"  

ques_discrep_string

/var/folders/rd/p9jwghh947z57nfswgxl_hf40000gq/T/ipykernel_59627/1279719067.py:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(row['field_label'], 'html.parser')


"\nThe following api-metadata question_text's are not in CRF_Schema_RedCap.question_text's:\n{redcap_variable : question_text}\n\n{'q3006a': 'Vessel present', 'q3011a': 'Vessel present', 'q3012a': 'Vessel present', 'q3013a': 'Vessel present', 'q3014a': 'Vessel present', 'q3015a': 'Vessel present', 'q3016a': 'Vessel present', 'q3017a': 'Vessel present', 'q3018a': 'Vessel present', 'q3019a': 'Vessel present', 'q3020a': 'Vessel present', 'q3021a': 'Vessel present', 'q3022a': 'Vessel present', 'q3023a': 'Vessel present', 'q3024a': 'Vessel present'}\n\n"

In [8]:
question_discreps

,field_name,field_label
124,q3006a,Vessel present
149,q3011a,Vessel present
154,q3012a,Vessel present
159,q3013a,Vessel present
164,q3014a,Vessel present
169,q3015a,Vessel present
174,q3016a,Vessel present
179,q3017a,Vessel present
184,q3018a,Vessel present
189,q3019a,Vessel present


In [4]:
api_questions

,field_name,field_label
116,q3001,Time point
117,q3002,Date of Imaging (mm/dd/yyyy)
118,q3003,Time of scan (hh:mm) 24-hour clock format
119,q3004,Was the T1 pre technically satisfactory?
120,q3004a,"If No, which of the sequences were not satisfa..."
...,...,...
222,q3028a,Purpose of data entry
223,q3028b,Final data verified
224,q3028c,Initials of reviewing lab personnel
225,q3028d,Date of reviewing lab personnel initials


In [5]:
schema_questions

0                                            Time point
1                          Date of Imaging (mm/dd/yyyy)
2             Time of scan (hh:mm) 24-hour clock format
3              Was the T1 pre technically satisfactory?
4     If No, which of the sequences were not satisfa...
5             Was the T1 post technically satisfactory?
6     If  "No", which of the sequences were not sati...
7                                        Vessel present
8                                           Enhancement
9                           T1 Intrinsic Hyperintensity
10                                 Surface Irregularity
11                                      Vessel present 
12                                    Surface Integrity
13                          Signature of Central Reader
14                     Date of Central Reader Signature
15                    Central Reader's General Comments
16                                Purpose of data entry
17                                  Final data v